# 🎶 Traducción de Letras de Canciones

## Objetivo
El objetivo de este proyecto es detectar el idioma de las canciones y traducir aquellas que no estén en inglés. Esto nos permitirá entrenar eficazmente un modelo de Roberta con canciones de distintos países y variedad. 🌍🎤

## Descripción del Proceso

1. **Importación de Módulos y Configuración Inicial** 📦
    - Importamos las bibliotecas necesarias como `pandas`, `langdetect`, `googletrans`, `deep_translator`, entre otras.
    - Configuramos los límites de tamaño de campo y rutas de los archivos.

2. **Carga y Filtrado de Datos** 📂
    - Cargamos los archivos CSV con las letras de canciones.
    - Filtramos las canciones que no están en inglés y aquellas que no tienen traducción.

3. **Detección de Idioma y Traducción** 🌐
    - Utilizamos `langdetect` para detectar el idioma de las letras.
    - Traducimos las letras no inglesas usando `googletrans` y `deep_translator`.

4. **Limpieza y Procesamiento de Texto** 🧹
    - Limpiamos las letras eliminando saltos de línea y espacios múltiples.
    - Dividimos textos largos para evitar errores en la traducción.

5. **Guardado de Resultados y Manejo de Errores** 💾
    - Guardamos las letras traducidas en un nuevo archivo CSV.
    - Registramos los errores encontrados durante el proceso de traducción.

6. **Verificación y Análisis** 🔍
    - Verificamos las traducciones y analizamos los resultados.
    - Fusionamos los datos traducidos con el archivo original para obtener un dataset completo.

## Herramientas Utilizadas
- `pandas` para manipulación de datos.
- `langdetect` para detección de idioma.
- `googletrans` y `deep_translator` para traducción automática.
- `tqdm` para mostrar el progreso del procesamiento.
- `transformers` para utilizar modelos avanzados de traducción como MarianMT.

## Resultados Esperados
Al final del proyecto, esperamos tener un dataset completo con letras de canciones traducidas al inglés, listo para ser utilizado en el entrenamiento de modelos de procesamiento de lenguaje natural (NLP). 🎯📊

## Cosas que se han probado
- `transformers` para utilizar modelos avanzados de traducción como MarianMT.

In [ ]:
import sys
import csv
import pandas as pd
from langdetect import detect
from deep_translator import GoogleTranslator
import re
from tqdm import tqdm  # Barra de progreso
import time
import os
from collections import deque  # Para mostrar solo los últimos 20 procesos

# 📌 Aumentar el límite del tamaño de campo
sys.setrecursionlimit(10000)
csv.field_size_limit(10**9)

# 📌 Configuración de archivos
file_path = os.path.join('data', 'filtrado_para_traduccion.csv')
output_file_path = os.path.join('data', 'traducidas.csv')
error_file_path = os.path.join('data', 'errors_log_traduccion.csv')
pause_time = 0.2  # Pausa para evitar bloqueos

# 📌 Inicializar el traductor
translator = GoogleTranslator(source="auto", target="en")

# 🔹 Función para limpiar texto antes de traducir
def clean_text(text):
    if pd.isna(text):
        return ""
    return re.sub(r'\s+', ' ', text.replace('\n', ' ').strip())

# 🔹 Función para dividir textos largas
def split_text(text, max_length=500):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# 📌 Cargar datos originales
data = pd.read_csv(file_path, encoding='utf-8', engine='python')

# 📌 Si existe `traducidas.csv`, cargarlo y limpiar duplicados
if os.path.exists(output_file_path):
    translated_data = pd.read_csv(output_file_path, encoding='utf-8', engine='python', on_bad_lines="skip")

    # 📌 Eliminar duplicados basados en `recording_id`
    translated_data = translated_data.drop_duplicates(subset=['recording_id'], keep='first')

    # 📌 Guardar el archivo limpio antes de fusionar
    translated_data.to_csv(output_file_path, index=False, encoding='utf-8')

    # 📌 Si `language` no existe en `traducidas.csv`, agregarlo
    if 'language' not in translated_data.columns:
        translated_data['language'] = pd.NA

    # 📌 Fusionar `data` con `translated_data`
    data = data.merge(translated_data[['recording_id', 'translated_lyrics', 'language']], on='recording_id', how='left', suffixes=('', '_prev'))
    data['translated_lyrics'] = data['translated_lyrics'].combine_first(data['translated_lyrics_prev'])
    data['language'] = data['language'].combine_first(data['language_prev'])  # Mantener el idioma original
    data.drop(columns=['translated_lyrics_prev', 'language_prev'], inplace=True)

print(f"🔍 Filas con traducción ya existente: {data['translated_lyrics'].notnull().sum()} de {len(data)}")

# 📌 Crear DataFrame para errores
errors = pd.DataFrame(columns=['index', 'lyrics', 'error_message'])

# 📌 Mantener solo los últimos 20 logs en la terminal
last_logs = deque(maxlen=20)

# 🔁 Procesar y traducir
for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="🔄 Traduciendo canciones"):
    try:
        if pd.notnull(row['translated_lyrics']) and str(row['translated_lyrics']).strip() != "":
            continue  # Saltar si ya está traducido

        lyrics_cleaned = clean_text(row['lyrics'])

        # 🔹 Evitar traducciones de textos vacíos
        if not lyrics_cleaned.strip():
            last_logs.append(f"⚠️ Letra vacía o inválida para {row['recording_id']}. Saltando...")
            continue

        # 🔹 Detectar idioma
        try:
            lang = row['language'] if pd.notnull(row['language']) else detect(lyrics_cleaned)
        except:
            lang = "unknown"  # Si langdetect falla, marcar como "desconocido"

        last_logs.append(f"🧐 Procesando {row['recording_id']} - Texto: {lyrics_cleaned[:50]}... 🔍 Idioma detectado: {lang}")

        # 🔥 Si la canción ya está en inglés, no traducirla
        if lang == "en":
            last_logs.append(f"✅ Canción {row['recording_id']} ya está en inglés. Saltando...")
            continue  # Saltar a la siguiente canción

        # 🔹 Intentar traducir hasta 3 veces si la traducción es vacía
        max_retries = 3
        translation = ""
        for attempt in range(max_retries):
            try:
                text_parts = split_text(lyrics_cleaned)
                translation = " ".join([translator.translate(part, source="auto") for part in text_parts])
                if translation.strip() != "":
                    break  # Si la traducción es válida, salir del loop
            except:
                last_logs.append(f"⚠️ Intento {attempt + 1} fallido. Reintentando...")
                time.sleep(1)

        if translation.strip() == "":
            last_logs.append(f"❌ No se pudo traducir {row['recording_id']}. Guardando como error.")
            errors = pd.concat([errors, pd.DataFrame({'index': [index], 'error_message': ["Traducción vacía después de reintentos"]})])
            continue

        data.at[index, 'translated_lyrics'] = translation
        data.at[index, 'language'] = lang  # Guardar el idioma detectado

        # 📌 Verificar antes de escribir para evitar duplicados en `traducidas.csv`
        translated_data = pd.read_csv(output_file_path, encoding='utf-8', engine='python', on_bad_lines="skip")
        if not ((translated_data['recording_id'] == row['recording_id']).any()):
            # 📌 Guardar traducción + idioma detectado en `traducidas.csv`
            pd.DataFrame({
                'recording_id': [row['recording_id']], 
                'translated_lyrics': [translation], 
                'language': [lang]  
            }).to_csv(output_file_path, mode='a', header=False, index=False, encoding='utf-8', sep=",", quoting=csv.QUOTE_MINIMAL)

        time.sleep(pause_time)

    except Exception as e:
        errors = pd.concat([errors, pd.DataFrame({'index': [index], 'error_message': [str(e)]})])

# 📌 Mostrar solo los últimos 20 logs
print("\n".join(last_logs))

# 📌 Guardar errores en archivo
errors.to_csv(error_file_path, index=False, encoding="utf-8")

print("✅ Proceso completado. Archivo traducido guardado.")


In [ ]:
import pandas as pd

# Ruta del archivo
file_path = os.path.join('data', 'final_idiomas_traducido.csv')

# Cargar el archivo CSV
df = pd.read_csv(file_path)

# Seleccionar las columnas de interés
columns_of_interest = ['artist_name', 'song_name', 'spotify_url', 'language', 'lyrics', 'translated_lyrics']

# Filtrar el DataFrame para mostrar solo las columnas de interés
df_filtered = df[columns_of_interest]

# Filtrar las filas donde el idioma no sea inglés ('en')
df_non_english = df_filtered[df_filtered['language'] != 'en']

# Mostrar las primeras filas del DataFrame filtrado
print("Primeras filas del DataFrame filtrado (idioma no inglés):")
print(df_non_english.sample())

# Mostrar las filas con valores nulos en las columnas de interés
print("\nFilas con valores nulos en las columnas de interés (idioma no inglés):")
print(df_non_english[df_non_english.isnull().any(axis=1)])

Primeras filas del DataFrame filtrado (idioma no inglés):
    artist_name        song_name  \
124       morat  como te atreves   

                                           spotify_url language  \
124  https://open.spotify.com/track/7M6CFruBrM5x7u0...       es   

                                                lyrics  \
124  hoy me pregunto que sera de ti te tuve cerca y...   

                                     translated_lyrics  
124  Today I wonder what it will be about you and n...  

Filas con valores nulos en las columnas de interés (idioma no inglés):
            artist_name                song_name  \
231       the saturdays  dont let me dance alone   
384           the hives   square one here i come   
445        say anything          high school low   
529         lara fabian     lamour existe encore   
546                rjd2            we come alive   
...                 ...                      ...   
4277          kvelertak         utrydd dei svake   
4312  katherine

In [ ]:
df_non_english.sample(20)

,artist_name,song_name,spotify_url,language,lyrics,translated_lyrics
61496,yelle,le grand saut,https://open.spotify.com/track/6ro7kKnnbgMVllC2ij5VCL,fr,cours ne te retourne pas le futur ouvre grand ses bras jour j pour faire le bon choix je monte a bord du vaisseau aujourdhui le grand saut cours ne te retourne pas le futur ouvre grand ses bras jour j pour faire le bon choix je monte a bord du vaisseau aujourdhui le grand saut le hasard sorganise et il suivra mes pas je mimmunise la pluie ne tombe pas sur moi mon hoodie en maille de fer lui resistera en un eclair je lui ouvre la voie le hasard sorganise et il suivra mes pas je mimmunise la pluie ne tombe pas sur moi mon hoodie en maille de fer lui resistera en un eclair je lui ouvre la voie cours ne te retourne pas le futur ouvre grand ses bras jour j pour faire le bon choix je monte a bord du vaisseau aujourdhui le grand saut le grand saut,lessons do not turn you around the future opens big day J to make the right choice I go up on board the ship today the big leap course does not turn you around the future opens big day D to make the right choice I go up on boardFrom the vessel today the big jump the sorganized chance and he will follow my steps I mimminate the rain does not fall on me my hoodie in iron mesh will resist him in an enclosure I opens the way to him the sorganized chance and he will follow my steps I mimmise theRain does not fall on me my hoodie in iron mesh will resist him in a light I opens the way to him does not turn you around the future opens his day jar to make the right choice I climb on board the ship today the big jumpThe big jump
74538,bisz,indygo,https://open.spotify.com/track/2XVzSdrpRpJN2PWeifnDet,pl,zwrotka 1 urodziłeś się królem pokaż wszystkim koronę kłów berło pięści wznieś jak księżyc nad morzem głów twój tron nosisz ze sobą ciągle na spodzie stóp nie szata czyni króla masz na sobie tylko spojrzeń rój niebo jest twoim dachem orszakiem ptaki o świcie błazny masy dla ciebie jak ty dla nich bo żyjesz poza ich pogonią i za czym za słoną cenę zapłaci każdy z nich gdy obudzi się za zasłoną nie ma nic poza spokojnym oddechem pierś bóstwa rozszerza się wszechświatem potem zapada się w punkt aby na nowo wziąć wdech i spokojnie się wzwyż wznieść cichy proces żaden wydumany big bang między ziemią a niebem jest człowiek z głową w chmurach stąpając twardo przewodzę przez moje życie tę siłę która czyni ciężkość lekką baranek który jest lwem jestem piękną bestią zmieszaj najjaśniejszy błękit z jak najgłębszą czernią refren wszyscy którzy wrócili z biegunów zawsze będą ze mną weź najciemniejszą czerń zmieszaj ją z jasnoniebieskim wyzwolony od zwycięstw i klęsk obserwuj ich zmierzch w indygo indygo indygo indygo indygo indygo indygo bridge gdy już nie wiesz dokąd iść słuchaj krwi która krąży w tobie daj się ponieść daj się ponieść w kolorze twojej krwi tkwi symbol i odpowiedź daj się ponieść daj się ponieść zwrotka 2 masz w sobie coś co możesz pokazać tylko wybranym wciąż jest nas zbyt mało by doprowadzić do zmiany lecz każdy z nas ma moc by zacząć powoli wsączać łzy w serce skały aby poruszyć monolit wiemy że możemy sprawić aby świat był lepszy i kto musi wziąć na barki trud nowi architekci my bo tu miejsca dla nas brak wciąż i gdy tylko chcesz otworzyć się słyszysz że masz się zamknąć lecz jeśli myśleli że mogą kazać ci cokolwiek tylko ze względu na twoją słabość biada im nauka dla ciebie i dla nich oby stąd wynikła nie wszyscy jeszcze się poddali chodnikowy wilk brat z otchłani bólu rozpaczy niewiary po szczyty nieba zrozumienia wolności i chwały stratowany fortuny kołem wiem jedno że to odległość pomiędzy górą a dołem jest pełnią refren outro gdy już nie wiesz dokąd iść słuchaj krwi która krąży w tobie daj się ponieść daj się ponieść w kolorze twojej krwi tkwi symbol i odpowiedź włóż koronę włóż koronę,"verse 1 You were born king. Show all the crown of fangs crown. ""Moon at the sea of ​​your head your throne.For them because you live beyond their pursuit a

#### 🤔💼 Otros intentos, pruebas etc...

In [ ]:
from langdetect import detect
from googletrans import Translator
import pandas as pd
import sys
import csv

# Aumentar el límite del tamaño de campo
sys.setrecursionlimit(10000)
csv.field_size_limit(10**9) 

# Configuración inicial para prueba
file_path = r'C:\Users\solan\Downloads\get_data_from_songs\data\final_idiomas.csv'  # Ruta al archivo original
output_file_path = r'C:\Users\solan\Downloads\get_data_from_songs\data\archivo_prueba_traducido.csv'  # Ruta de salida para la prueba
error_file_path = r'C:\Users\solan\Downloads\get_data_from_songs\data\errors_log_prueba.csv'  # Ruta del archivo de errores para la prueba
translator = Translator()


# Cargar el archivo completo
data = pd.read_csv(file_path, encoding='utf-8', engine='python')

# Filtrar las filas que no están en inglés
non_english_rows = []
for index, row in data.iterrows():
    if pd.isna(row['language']) or row['language'] != 'en':
        non_english_rows.append(row)
    if len(non_english_rows) >= 100:  # Seleccionar solo las primeras 100
        break

# Crear un DataFrame con las primeras 100 filas que no son inglés
test_data = pd.DataFrame(non_english_rows)

# Añadir columna para letras traducidas si no existe
if 'translated_lyrics' not in test_data.columns:
    test_data['translated_lyrics'] = None

# Crear un DataFrame vacío para almacenar errores
errors = pd.DataFrame(columns=['index', 'lyrics', 'error_message'])

# Procesar y traducir
for index, row in test_data.iterrows():
    try:
        # Detectar idioma si no está indicado
        lang = row['language'] if pd.notnull(row['language']) else detect(row['lyrics'])
        
        # Traducir solo si no está en inglés
        if lang != 'en':
            translation = translator.translate(row['lyrics'], src=lang, dest='en')
            test_data.at[index, 'translated_lyrics'] = translation.text

    except Exception as e:
        # Registrar el error en el DataFrame de errores
        errors = pd.concat([errors, pd.DataFrame({'index': [index], 
                                                  'lyrics': [row['lyrics']], 
                                                  'error_message': [str(e)]})])

# Guardar los resultados de la prueba
test_data.to_csv(output_file_path, index=False, encoding='utf-8')
errors.to_csv(error_file_path, index=False, encoding='utf-8')
print("Prueba completada. Resultados y errores guardados.")



In [ ]:
import pandas as pd

# 📂 Ruta del archivo original
file_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\functions b\df_lyrics_faltan_traduc.csv"
output_filtered_path = r"C:\Users\solan\Downloads\get_data_from_songs\data\filtrado_para_traduccion.csv"

# 🔹 Cargar el archivo original
df = pd.read_csv(file_path, low_memory=False)

# 🔹 Filtrar: donde `language` no sea "en" y `translated_lyrics` esté vacío o NaN
df_filtered = df[(df['language'] != "en") & (df['translated_lyrics'].isna())]

# 🔹 Guardar el nuevo CSV para usar en la traducción
df_filtered.to_csv(output_filtered_path, index=False, encoding="utf-8")

print(f"✅ Archivo filtrado guardado en: {output_filtered_path}")


✅ Archivo filtrado guardado en: C:\Users\solan\Downloads\get_data_from_songs\data\filtrado_para_traduccion.csv


In [ ]:
import sys
import csv
import pandas as pd
from langdetect import detect
from googletrans import Translator
import re
from tqdm import tqdm  # Importar tqdm para barra de progreso

# Aumentar el límite del tamaño de campo
sys.setrecursionlimit(10000)
csv.field_size_limit(10**9)

# Configuración inicial
file_path = r'C:\Users\solan\Downloads\get_data_from_songs\data\filtrado_para_traduccion.csvv'  # Ruta al archivo original
output_file_path = r'C:\Users\solan\Downloads\get_data_from_songs\data\df_scrap_traducido.csv'  # Ruta de salida
error_file_path = r'C:\Users\solan\Downloads\get_data_from_songs\data\errors_log_traduccion.csv'  # Archivo de errores
batch_size = 1000  # Guardar cada 1,000 filas procesadas
translator = Translator()

# Función para limpiar texto
def clean_text(text):
    if pd.isna(text):
        return ""
    # Reemplazar saltos de línea y espacios múltiples por un solo espacio
    text = re.sub(r'\s+', ' ', text.replace('\n', ' ').strip())
    return text

# Cargar datos
data = pd.read_csv(file_path, encoding='utf-8', engine='python')

# Añadir columna para letras traducidas si no existe
if 'translated_lyrics' not in data.columns:
    data['translated_lyrics'] = None

# Crear un DataFrame vacío para almacenar errores
errors = pd.DataFrame(columns=['index', 'lyrics', 'error_message'])

# Procesar y traducir con barra de progreso
for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Procesando letras de canciones"):
    try:
        # Verificar si ya existe una traducción
        if pd.notnull(row['translated_lyrics']):
            continue

        # Limpiar la letra antes de procesar
        lyrics_cleaned = clean_text(row['lyrics'])

        # Detectar idioma si no está indicado
        lang = row['language'] if pd.notnull(row['language']) else detect(lyrics_cleaned)

        # Traducir solo si no está en inglés
        if lang != 'en' and lyrics_cleaned:
            translation = translator.translate(lyrics_cleaned, src=lang, dest='en')
            data.at[index, 'translated_lyrics'] = translation.text

        # Guardar cada batch de filas procesadas
        if index % batch_size == 0:
            data.to_csv(output_file_path, index=False, encoding='utf-8')
            errors.to_csv(error_file_path, index=False, encoding='utf-8')

    except Exception as e:
        # Registrar el error en el DataFrame de errores
        errors = pd.concat([errors, pd.DataFrame({'index': [index], 
                                                  'lyrics': [row['lyrics']], 
                                                  'error_message': [str(e)]})])

# Guardar archivo final
data.to_csv(output_file_path, index=False, encoding='utf-8')
errors.to_csv(error_file_path, index=False, encoding='utf-8')
print("Proceso completado y archivo final guardado.")


Opcion deep trasnlator

In [ ]:
from deep_translator import GoogleTranslator

# Crear un traductor de español a inglés
translator = GoogleTranslator(source="auto", target="en")

# Traducir una frase de prueba
translation = translator.translate("Hola mundo")
print(translation)  # Output: "Hello world"



Hello world


In [ ]:
translated_data = translated_data.drop_duplicates(subset=['recording_id'])

data = data.merge(translated_data[['recording_id', 'translated_lyrics']], on='recording_id', how='left', suffixes=('', '_prev'))
data['translated_lyrics'] = data['translated_lyrics'].combine_first(data['translated_lyrics_prev'])
data.drop(columns=['translated_lyrics_prev'], inplace=True)


In [ ]:
if os.path.exists(output_file_path):
    translated_data = pd.read_csv(output_file_path, encoding='utf-8', engine='python', on_bad_lines="skip")
    print("📌 Columnas en traducidas.csv:", translated_data.columns)


NameError: name 'os' is not defined

In [ ]:
df_check = pd.read_csv(output_file_path)
print(df_check.head(10))  # Ver 10 primeras filas
print(f"Total de traducciones guardadas: {df_check['translated_lyrics'].notnull().sum()}")


               artist_name           song_name  \
0                     ceza  bir minik mikrofon   
1                 enjambre         sanguijuela   
2               ana tijoux                sube   
3               marcelo d2       malandro rife   
4                onda vaga           marineros   
5              avi buffalo   five little sluts   
6               alligatoah          der zensor   
7          no te va gustar   con la misma vara   
8                   tarkan               kayıp   
9  antony and the johnsons              flétta   

                           recording_id  danceable  not_danceable   male  \
0  0198cdde-58fb-47bd-a5e9-6878bbcc7f48      0.980          0.020  0.233   
1  040bb6bd-dd6e-4203-9df4-84832e70efcc      0.707          0.293  0.633   
2  043b6f7b-6234-4b21-ac9a-fecfaf701f24      0.918          0.082  0.238   
3  089be47b-8cf5-4343-9d31-fa157f519d54      0.847          0.153  0.100   
4  08e13e69-d662-45ed-a0de-c047e3665e82      0.731          0.269  0.

C:\Users\solan\AppData\Local\Temp\ipykernel_5688\2858236912.py:1: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  df_check = pd.read_csv(output_file_path)


In [ ]:
print(f"🔍 Filas con traducción ya existente: {num_translated} de {len(data)}")


🔍 Filas con traducción ya existente: 1650 de 37106


In [ ]:
# Revisar cuántos valores traducidos había en el archivo antes de fusionar
print(f"Antes de fusionar, traducciones en archivo guardado: {translated_data['translated_lyrics'].notnull().sum()}")

# Revisar si después de merge() realmente se añadieron los valores
print(f"Después de fusionar, traducciones en el dataset actual: {data['translated_lyrics'].notnull().sum()}")

# Revisar si `recording_id` tiene coincidencias exactas entre ambos datasets
common_ids = data['recording_id'].isin(translated_data['recording_id']).sum()
print(f"Número de recording_id coincidentes entre datasets: {common_ids} de {len(data)}")

# Mostrar algunas filas para ver qué está pasando
print(data[['recording_id', 'translated_lyrics']].head(10))


Antes de fusionar, traducciones en archivo guardado: 1331
Después de fusionar, traducciones en el dataset actual: 1709
Número de recording_id coincidentes entre datasets: 37106 de 37106
                           recording_id  \
0  0198cdde-58fb-47bd-a5e9-6878bbcc7f48   
1  040bb6bd-dd6e-4203-9df4-84832e70efcc   
2  043b6f7b-6234-4b21-ac9a-fecfaf701f24   
3  089be47b-8cf5-4343-9d31-fa157f519d54   
4  08e13e69-d662-45ed-a0de-c047e3665e82   
5  0933b7b9-88f6-4537-ad65-39a6719f10f3   
6  0a9370ee-4384-4d0e-8615-451830b66e6f   
7  0b1ef0ee-351a-4005-8997-742e60b4133c   
8  0c13cab6-1e86-4794-9591-e00e0d179251   
9  0c2ebf24-0ad1-4906-a2ae-3777d1820e7e   

                                   translated_lyrics  
0  verse 1 sapr sapr falls on the ground there we...  
1  because of the heat I always said that if I do...  
2  lyrics of sube ft invincible intro panic room ...  
3  rogue rogue rogue rogue rogue rogue rogue rogu...  
4  we like the boat we like the wind although som...  
5  cacaro 

In [ ]:
print(translated_data[['recording_id', 'translated_lyrics']].dropna().head(20))
print(f"Total de traducciones guardadas en el archivo: {translated_data['translated_lyrics'].notnull().sum()}")


                            recording_id  \
0   0198cdde-58fb-47bd-a5e9-6878bbcc7f48   
1   040bb6bd-dd6e-4203-9df4-84832e70efcc   
2   043b6f7b-6234-4b21-ac9a-fecfaf701f24   
3   089be47b-8cf5-4343-9d31-fa157f519d54   
4   08e13e69-d662-45ed-a0de-c047e3665e82   
5   0933b7b9-88f6-4537-ad65-39a6719f10f3   
6   0a9370ee-4384-4d0e-8615-451830b66e6f   
7   0b1ef0ee-351a-4005-8997-742e60b4133c   
8   0c13cab6-1e86-4794-9591-e00e0d179251   
9   0c2ebf24-0ad1-4906-a2ae-3777d1820e7e   
10  0f2a7497-ed10-48a8-a6eb-e0f2b72e96ec   
11  1065cb20-7451-436c-9490-82c6f97176f2   
12  106d4a91-cf27-41c7-b04a-b35539f0cc61   
13  12a621d9-020c-4397-9014-88c8772149f7   
14  14612e27-4c6d-4d58-aff1-8465ec85200b   
15  16726338-44a5-45f1-9b09-88145374c4c0   
16  1698666c-0010-431d-a702-ef210d50e10c   
18  1b944efe-dbfb-4051-b53b-7e182193a0cf   
19  1bf8b086-0c8e-47b6-9c15-e3ad7352948b   
20  1e9b3680-f8f2-471f-8ad7-81de92c92188   

                                    translated_lyrics  
0   verse 1 sapr sa

otro modelo - no

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from langdetect import detect
import pandas as pd

# Configuración del modelo MarianMT
model_name = "Helsinki-NLP/opus-mt-es-en"  # Cambia el modelo según el idioma (ej. `es-en`, `fr-en`)
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Función para traducir usando MarianMT
def translate_marian(texts, source_lang="es", target_lang="en"):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Cargar datos del archivo
file_path = r'C:\ruta\a\final_idiomas.csv'  # Ruta del archivo original
output_file_path = r'C:\ruta\a\final_idiomas_traducido_marian.csv'  # Archivo de salida
data = pd.read_csv(file_path, encoding='utf-8')

# Añadir columna para letras traducidas si no existe
if 'translated_lyrics' not in data.columns:
    data['translated_lyrics'] = None

# Procesar y traducir
for index, row in data.iterrows():
    try:
        # Verificar si ya existe una traducción
        if pd.notnull(row['translated_lyrics']):
            continue
        
        # Detectar idioma si no está especificado
        lyrics = row['lyrics']
        lang = row['language'] if pd.notnull(row['language']) else detect(lyrics)

        # Traducir solo si el idioma no es inglés
        if lang != 'en' and pd.notnull(lyrics):
            translated_text = translate_marian([lyrics])[0]
            data.at[index, 'translated_lyrics'] = translated_text

    except Exception as e:
        print(f"Error al traducir en la fila {index}: {e}")

# Guardar archivo final con traducciones
data.to_csv(output_file_path, index=False, encoding='utf-8')
print(f"Traducciones completadas. Archivo guardado en {output_file_path}.")


In [ ]:
pd.set_option('display.max_colwidth', None)

NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
import csv

# 📂 Rutas de los archivos
final_url_path = r"C:\Users\solan\Downloads\get_data_from_songs\data\final_url6.csv"
translated_lyrics_path = r"C:\Users\solan\Downloads\get_data_from_songs\data\final_idiomas_traducido.csv"
output_path = r"C:\Users\solan\Downloads\get_data_from_songs\data\final_with_translations1faltanscrap.csv"

# 1️⃣ Cargar los archivos CSV
df_final = pd.read_csv(final_url_path, low_memory=False)
df_translations = pd.read_csv(translated_lyrics_path, low_memory=False)

# 2️⃣ Verificar que 'recording_id' existe en ambos
if 'recording_id' not in df_final.columns or 'recording_id' not in df_translations.columns:
    raise KeyError("❌ ERROR: 'recording_id' no se encuentra en ambos datasets.")

# 3️⃣ Seleccionar solo la columna 'recording_id' y 'translated_lyrics' para fusionar
df_translations = df_translations[['recording_id', 'translated_lyrics']]

# 4️⃣ Reemplazar saltos de línea por un espacio simple y limpiar espacios extra
df_translations['translated_lyrics'] = df_translations['translated_lyrics'].astype(str).apply(lambda x: " ".join(x.split()))

# 5️⃣ Hacer el merge por 'recording_id'
df_merged = df_final.merge(df_translations, on='recording_id', how='left')

# 6️⃣ Guardar el archivo final con formato correcto
df_merged.to_csv(output_path, index=False, quoting=csv.QUOTE_ALL, lineterminator='\n')

print(f"✅ Archivo final guardado con las traducciones en: {output_path}")


✅ Archivo final guardado con las traducciones en: C:\Users\solan\Downloads\get_data_from_songs\data\final_with_translations1faltanscrap.csv


In [ ]:
df_non_english.head()